In [ ]:
import io

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

The aim of this notebook is to parse the PDF files and extract abstracts' text to plain text files.

**NOTE**: running this notebook requires the installation of the pdfminer2 package. 

In [ ]:
#pdf_file = 'papers/Zuffi_Lions_and_Tigers_CVPR_2018_paper.pdf'
pdf_file = 'papers/Yu_DoubleFusion_Real-Time_Capture_CVPR_2018_paper.pdf'

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

text = ' '.join(convert_pdf_to_txt(pdf_file).split('\n')).replace('- ','')

# TODO: it should be enough to get what's in between abstract and 1. Introduction, but sometimes
# specially if there is a figure at the beginning, this may not work. 
# For each pdf extract what it is between abstract and introduction, and take a look at the distribution
# of the length of abstracts to detect weird cases

text